In [2]:
# //****************************************************************************************
# //*** Set Working Directory to thinkstats folder.
# //*** This pseudo-relative path call should work on all Stoneburner localized projects. 
# //****************************************************************************************

import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3

output_width = 1000
#output_width = 80 #//*** Normal Output width

#//*** Normal Output width
pd.set_option("display.width", output_width)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
#//*** Returns True if database connection is open. Otherise returns False. I liberally borrowed (ie Stole)
#//*** from the Inter-webs
def chk_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False
    
db_filename = 'z_sup_wk10_11_petsdbz'

#//*** Verify database instance exists
if os.path.exists(db_filename) == False:
    #//*** Throw a warning if databse does not exist
    print(f"Trouble Connecting to database: {db_filename}")
else:
    #//*** Start a database connection
    conn = sqlite3.connect(db_filename)
    
    #//*** Verify database connection
    if chk_conn(conn):
        print("Database Connected....")    

Database Connected....


In [4]:
### Write your code below this comment
cursor = conn.cursor()

#//*** Get a list of all the tables in the database.
#//*** Print a list of the columns in each table for reference
pet_cols = []
person_cols = []

for results in conn.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    table = results[0]
    print("========================")
    print(f"Table: {table}")
    print("========================")
    print("Columns:")


    #//*** Read from the table, this modfies the cursor object, which will provide column names
    for col in cursor.execute(f'SELECT * FROM {table}').description:
        print(f"\t{col[0]}")
            
    print("")


Table: persons
Columns:
	id
	first_name
	last_name
	age
	city
	zip_code

Table: pets
Columns:
	owner_id
	pet_name
	pet_type
	treatment_done



In [39]:
query = """
        SELECT * FROM persons
"""
print(f"Persons Rows: {len(pd.read_sql(query,con=conn))}")
print(pd.read_sql(query,con=conn).head(10))

Persons Rows: 100
   id first_name last_name  age        city  zip_code
0   1      Erica      None   22  south port   2345678
1   2      Jordi      None   73   east port    123456
2   3    Chasity      None   70    new port  76856785
3   4      Gregg      None   31    new port  76856785
4   5       Tony  Lindgren    7   west port   2345678
5   6       Cary      None   73    new port  76856785
6   7       Gabe  Thompson   54    new port   9756543
7   8  Francisca      None   14   west port    123456
8   9    Katelyn    Torphy   49   east port   9756543
9  10    Raleigh      None   68    new port   2345678


In [40]:
query = """
        SELECT * FROM pets
"""
print(f"Pets Rows: {len(pd.read_sql(query,con=conn))}")
print(pd.read_sql(query,con=conn).head(10))

Pets Rows: 150
   owner_id pet_name  pet_type  treatment_done
0        57     mani       1.0               0
1        80   tamari       NaN               0
2        25     raba       NaN               0
3        27     olga       NaN               0
4        60     raba       NaN               0
5        37     dara       1.0               0
6        33   chegal       1.0               0
7        16     dara       NaN               0
8       100   chegal       NaN               0
9        46     raba       NaN               1


In [42]:
query = """
        SELECT * FROM persons
        LEFT OUTER JOIN pets ON persons.id = pets.owner_id 
"""
print(f"Results: {len(pd.read_sql(query,con=conn))} rows")
print(pd.read_sql(query,con=conn).head(10))

Results: 171 rows
   id first_name last_name  age        city  zip_code  owner_id pet_name  pet_type  treatment_done
0   1      Erica      None   22  south port   2345678       1.0     raba       NaN             1.0
1   2      Jordi      None   73   east port    123456       2.0    gimir       1.0             0.0
2   2      Jordi      None   73   east port    123456       2.0     mani       1.0             0.0
3   2      Jordi      None   73   east port    123456       2.0   tamari       NaN             1.0
4   3    Chasity      None   70    new port  76856785       3.0   tamari       NaN             1.0
5   4      Gregg      None   31    new port  76856785       NaN     None       NaN             NaN
6   5       Tony  Lindgren    7   west port   2345678       5.0     dara       NaN             0.0
7   5       Tony  Lindgren    7   west port   2345678       5.0     dara       1.0             0.0
8   5       Tony  Lindgren    7   west port   2345678       5.0    fenga       NaN         

In [41]:
query = """
        SELECT * FROM persons
        INNER JOIN pets ON persons.id = pets.owner_id 
"""
print(f"Results: {len(pd.read_sql(query,con=conn))} rows")
print(pd.read_sql(query,con=conn).head(10))

Results: 150 rows
   id first_name last_name  age        city  zip_code  owner_id pet_name  pet_type  treatment_done
0   1      Erica      None   22  south port   2345678         1     raba       NaN               1
1   2      Jordi      None   73   east port    123456         2    gimir       1.0               0
2   2      Jordi      None   73   east port    123456         2     mani       1.0               0
3   2      Jordi      None   73   east port    123456         2   tamari       NaN               1
4   3    Chasity      None   70    new port  76856785         3   tamari       NaN               1
5   5       Tony  Lindgren    7   west port   2345678         5     dara       NaN               0
6   5       Tony  Lindgren    7   west port   2345678         5     dara       1.0               0
7   5       Tony  Lindgren    7   west port   2345678         5    fenga       NaN               0
8   6       Cary      None   73    new port  76856785         6     deru       1.0         

In [9]:
query = """
        SELECT * FROM persons
        NATURAL JOIN pets 
"""
print(len(pd.read_sql(query,con=conn)))
print(pd.read_sql(query,con=conn).head(10))

15000
   id first_name last_name  age        city  zip_code  owner_id pet_name  pet_type  treatment_done
0   1      Erica      None   22  south port   2345678        57     mani       1.0               0
1   1      Erica      None   22  south port   2345678        80   tamari       NaN               0
2   1      Erica      None   22  south port   2345678        25     raba       NaN               0
3   1      Erica      None   22  south port   2345678        27     olga       NaN               0
4   1      Erica      None   22  south port   2345678        60     raba       NaN               0
5   1      Erica      None   22  south port   2345678        37     dara       1.0               0
6   1      Erica      None   22  south port   2345678        33   chegal       1.0               0
7   1      Erica      None   22  south port   2345678        16     dara       NaN               0
8   1      Erica      None   22  south port   2345678       100   chegal       NaN               0
9   

In [26]:
query = """
        SELECT * FROM persons
        CROSS JOIN pets 
"""

df = pd.read_sql(query,con=conn)
slices = 5
interval = int(len(df)/slices)
print(f"Total Cross Join Values: {len(df)}")
for x in range(0,slices):
    print(df[x*interval:x*interval+5])

Total Cross Join Values: 15000
   id first_name last_name  age        city  zip_code  owner_id pet_name  pet_type  treatment_done
0   1      Erica      None   22  south port   2345678        57     mani       1.0               0
1   1      Erica      None   22  south port   2345678        80   tamari       NaN               0
2   1      Erica      None   22  south port   2345678        25     raba       NaN               0
3   1      Erica      None   22  south port   2345678        27     olga       NaN               0
4   1      Erica      None   22  south port   2345678        60     raba       NaN               0
      id first_name last_name  age        city  zip_code  owner_id pet_name  pet_type  treatment_done
3000  21      Berry      None   22  south port   2345678        57     mani       1.0               0
3001  21      Berry      None   22  south port   2345678        80   tamari       NaN               0
3002  21      Berry      None   22  south port   2345678        25   